In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 43 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 7.5 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595725 sha256=5999ff97d44a459e73c55d877f4158d5ba4d6e1e9a21889402eee9901311f289
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 9.7 MB/s 
     |████████████████████████████████| 769 kB 9.7 MB/s 
     |████████████████████████████████| 3.0 MB 56.0 MB/s 
     |████████████████████████████████| 895 kB 60.2 MB/s 


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-m_xqugas
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-m_xqugas
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12770 sha256=e935119055b59188fe5578ea8c75b1acc0d116b6271d0b3830261627f07e9564
  Stored in directory: /tmp/pip-ephem-wheel-cache-i5n0i7rf/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
# !wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
# !wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

In [ ]:
# dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
# dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [ ]:
import numpy as np
import pandas as pd
data=pd.read_excel('/content/공모전_제공_데이터(1차).xlsx')

In [ ]:
map={'칭찬>고객서비스>상담원':0,'칭찬>고객서비스>상담시스템':1,'칭찬>고객서비스':2,'불만>고객서비스>상담원':3,
     '불만>고객서비스>상담시스템':4,'불만>고객서비스':5,'칭찬>삼성카드>혜택':6,'칭찬>삼성카드>할부금융상품':7,
     '칭찬>삼성카드>커뮤니티서비스':8,'칭찬>삼성카드>카드이용/결제':9,'칭찬>삼성카드>카드상품':10,
     '칭찬>삼성카드>청구입금':11,'칭찬>삼성카드>심사/한도':12,'칭찬>삼성카드>생활편의서비스':13,
     '칭찬>삼성카드>상담/채널':14,'칭찬>삼성카드>리스렌탈상품':15,'칭찬>삼성카드>라이프서비스':16,
     '칭찬>삼성카드>금융상품':17,'칭찬>삼성카드>고객정보관리':18,'칭찬>삼성카드>가맹점매출/승인':19,
     '칭찬>삼성카드>가맹점대금':20,'칭찬>삼성카드>가맹점계약':21,'칭찬>삼성카드':22,'칭찬>기타':23,'불만>삼성카드>혜택':24,
     '불만>삼성카드>할부금융상품':25,'불만>삼성카드>커뮤니티서비스':26,'불만>삼성카드>카드이용/결제':27,'불만>삼성카드>카드상품':28,
     '불만>삼성카드>청구입금':29,'불만>삼성카드>심사/한도':30,'불만>삼성카드>생활편의서비스':31,'불만>삼성카드>상담/채널':32,
     '불만>삼성카드>리스렌탈상품':33,'불만>삼성카드>라이프서비스':34,'불만>삼성카드>금융상품':35,'불만>삼성카드>고객정보관리':36,
     '불만>삼성카드>가맹점매출/승인':37,'불만>삼성카드>가맹점대금':38,'불만>삼성카드>가맹점계약':39,'불만>삼성카드':40,
     '불만>기타':41,'중립':42,'폐기':43}

In [ ]:
data=data[['발화','최종분류(우선순위 가장 높은것 선택)']]
data=data.rename(columns={'발화':'sentence','최종분류(우선순위 가장 높은것 선택)':'label'})

data['label']=data['label'].map(map)

In [ ]:
X_train

703                                         친절 상담 감사드립니다
311    좋았습니다. 다만 보험 전화 좀 그만하셨으면 좋겠습니다. 이미 가입한 보험이 있는데...
722                                           친절에 감사합니다.
629                                         친절한 상담 감사드려요
0                                       상담직원과 빠른연결했슴좋겠어요
                             ...                        
106                                        통화대기시간 이 너무길다
270    고객이 모르는 것에 대해서 물어보면 친절하게 응대해주세요. 말투가 무시한다는 느낌을...
860                                 엘지유플러스 직원의친절함 적극성!!!
435                  상담원과 연결하기가 복잡합니다 편리하게 상담원과 연결을 원합니다
102                                        상담원연결시간이너무길고.
Name: sentence, Length: 670, dtype: object

(703                                         친절 상담 감사드립니다
 311    좋았습니다. 다만 보험 전화 좀 그만하셨으면 좋겠습니다. 이미 가입한 보험이 있는데...
 722                                           친절에 감사합니다.
 629                                         친절한 상담 감사드려요
 0                                       상담직원과 빠른연결했슴좋겠어요
                              ...                        
 106                                        통화대기시간 이 너무길다
 270    고객이 모르는 것에 대해서 물어보면 친절하게 응대해주세요. 말투가 무시한다는 느낌을...
 860                                 엘지유플러스 직원의친절함 적극성!!!
 435                  상담원과 연결하기가 복잡합니다 편리하게 상담원과 연결을 원합니다
 102                                        상담원연결시간이너무길고.
 Name: sentence, Length: 670, dtype: object, 703     0
 311    36
 722     0
 629     0
 0       4
        ..
 106     4
 270     3
 860    23
 435    32
 102     4
 Name: label, Length: 670, dtype: int64)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['sentence'],data['label'],test_size=0.33, random_state=42)

In [ ]:
for i in X_train:
    print(i)
    break

친절 상담 감사드립니다


In [ ]:
X_train.index

Int64Index([703, 311, 722, 629,   0, 316, 706, 547, 872, 532,
            ...
            121, 614,  20, 700,  71, 106, 270, 860, 435, 102],
           dtype='int64', length=670)

In [ ]:
dataset_train=[]
dataset_test=[]

for i in X_train.index:
    dataset_train.append((X_train[i],y_train[i]))

for i in X_test.index:
    dataset_test.append((X_test[i],y_test[i]))

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
# max_len = 64
max_len=128
# batch_size = 64
batch_size=32
warmup_ratio = 0.1
# num_epochs = 5
num_epochs=20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                #  num_classes=2
                 num_classes=44,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 3.848464012145996 train acc 0.0

epoch 1 train acc 0.08839285714285713


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.27159090909090905


...


epoch 18 test acc 0.8289772727272727


epoch 19 batch id 1 loss 0.3945544958114624 train acc 0.90625

epoch 19 train acc 0.9567460317460317



epoch 19 test acc 0.8261363636363637


epoch 20 batch id 1 loss 0.3981163203716278 train acc 0.9375

epoch 20 train acc 0.9551587301587302



epoch 20 test acc 0.8261363636363637
